In [1]:
import geopandas as gpd
import shapely.geometry
from shapely import wkt
import pandas as pd
import ast
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon, Polygon
from shapely.validation import make_valid

In [ ]:
country_side_data = pd.read_csv("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/new_defra_data/final_database/LR_country_Side_common_ids/LR_and_country_side_Data_for_analysis.csv")
country_side_data['geometry'] = country_side_data['geometry'].apply(wkt.loads)
country_side_data = gpd.GeoDataFrame(country_side_data)
country_side_data = country_side_data.set_crs(27700, allow_override=True)
country_side_data

In [ ]:
country_owner = country_owner[[  'ORG_NAME', 
       'CSREF', 'SUM_GIS_HA','calculated', 'Area_hc_st', 'crop_maxA','Area(hec)', 'geometry']]

country_owner

In [ ]:
ord_data = gpd.read_file("/Users/smit0225/Documents/Oxford/Agriculture/Land_Registry/filtered_fields/OSMM_england/Ord_fields_only_england_final.shp")
ord_data = ord_data[['primary_ke', 'fid', 
        'calculated',  'Area(hec)', 'geometry']]
ord_data

In [ ]:
a = gpd.sjoin(ord_data,country_owner, predicate='intersects')
a = a.reset_index(drop=True)


geo_idx = 0
for idx, col_n in enumerate(a.columns):
    if col_n=='geometry':
        geo_idx = idx
# Apply this to the geometries that might be invalid
a['geometry'] = a['geometry'].apply(lambda geom: make_valid(geom) if not geom.is_valid else geom)
country_owner['geometry'] = country_owner['geometry'].apply(lambda geom: make_valid(geom) if not geom.is_valid else geom)

a['L_O_area_prc'] = a.apply(lambda row: (row[geo_idx].intersection(country_owner.iloc[row[geo_idx+1]].geometry).area / row[geo_idx].area) * 100, axis=1)
a = a.reset_index(drop=True)
idx = a.groupby('fid')['L_O_area_prc'].idxmax()
final_data = a.loc[idx]
final_data = final_data.reset_index(drop=True)
final_data

In [ ]:
final_data['L_O_area_olap'] = (final_data['L_O_area_prc'] *final_data['Area(hec)'] )/100
final_data

In [ ]:

condition = (final_data['L_O_area_olap'] > 0.05) & (final_data['L_O_area_prc'] >= 10)

final_data = final_data[condition]
final_data

In [ ]:
final_data.to_file("./clean_CS_Ord/CS_ORD_clean.shp")